# LunaSolve

## Solving real-world optimization challenges efficiently


LunaSolve is a service dedicated to optimizing business solutions by offering tailored software and hardware options for tackling optimization challenges. With our user-centric approach, you can create optimization instances using a documented high-level specification, or upload problems using industry-standard modeling formats. Once submitted, our AI-driven recommendation engine suggests suitable software and hardware based on your specific use case. We then leverage quantum computers to handle the challenge, process the data, and deliver the solution. Through post-processing techniques, we convert raw quantum data into a human-readable format before sharing the response with you.

When using LunaSolve, you typically follow three steps:

- **Transformation:** Go from your problem definition to a (quantum-ready) mathematical format
- **Recommendation:** Get a suggestion on the best combination of software and hardware for your use case
- **Solve:** Solve the problem on quantum or classical hardware, or a combination of both.

To get started with LunaSolve, refer to the [Get Started page](https://docs.aqarios.com/get-started) for instructions. Once you've set the foundation, you can directly dive into the service and start solving your optimization challenges.

In [ ]:
# Load the luna package
from luna import Luna

# Create a Luna object and set your credentials
luna = Luna(credentials="credentials.json")

# Create your LunaSolve session
ls = luna.start_LunaSolve()

## Transformation: How to create optimization problem instances

To start solving an optimization problem, the first step is to define which problem you want to solve in the first place. You have two different approaches how to formulate the problem: You can either choose an optimization problem from our [use case library](https://docs.aqarios.com/use-cases) or directly provide your use case in a mathematical modeling format. At the moment, we only support QUBOs as predefined modeling formats, but other modelling languages will be included in future versions.

### Access our use case library

We provide users with a built-in library of (currently) more than 40 different optimization problems which can be configured using human-readable data formats such as graphs or lists. These are then automatically transformed into appropriate mathematical representations for optimization algorithms and quantum hardware.

The list of currently available optimizations can be retrieved from our [use case library](https://docs.aqarios.com/use-cases). Here, you can also find detailed information on how to specify each problem formulation, i.e. which parameters you need to provide for each optimization problem and which format they should be in.

As an example, let's say you want to solve a Traveling Salesperson Problem (TSP). From our use case library, you can find all information about the use case, including a description, a link to a detailed description of the mathematical model, and any parameters necessary to create a problem instance. Here, you will see that to create an instance of TSP, the only data you need to provide is a graph. This graph can either be a nested dictionary or can be created from `networkx`. We recommend the latter approach.

In [ ]:
import networkx as nx

# Create an empty graph using networkx
graph = nx.Graph()

# Define the nodes of our TSP instance
nodes = ['A', 'B', 'C', 'D']

# Add the nodes to our graph
graph.add_nodes_from(nodes)

# Define the distances between each node
edges = [('A', 'B', 10), 
         ('A', 'C', 9), 
         ('A', 'D', 13), 
         ('B', 'C', 6), 
         ('B', 'D', 21), 
         ('C', 'D', 15)]

# Add the distances as the edges in our graph
graph.add_weighted_edges_from(edges)

With this, we have defined our graph and provided all the necessary specifications. Now, let's send our graph to Luna and generate the TSP instance.

In [ ]:
# Convert the networkx graph to a digestible format for Luna
graph_data = nx.to_dict_of_dicts(graph)

# Define that we want to create a TSP with the corresponding graph data
tsp_instance =  {
    "name": "TSP",
    "graph": graph_data
}

# Create a TSP instance and retrieve the corresponding ID
tsp_id = luna.create_optimization(instance=tsp_instance)

When creating an instance of an optimization problem, Luna will store this problem in your account's database. This way, you can access the instance for further calculations or revisit it at a later time.

### Uploading custom optimization problems

We also offer the ability to solve instances of problem classes that are not included in our use case library. To achieve this, you can send the mathematical model of your optimization problem to Luna. Currently, only QUBOs are supported, but we plan to include other mathematical formats in future versions. QUBOs can be provided as nested lists of floats, and all the same functionalities available to optimization problems are accessible for problems formulated as QUBOs as well.

In [ ]:
qubo = [
    [4, 0, 0, 0, -2],
    [0, -2, 0, 0, 0],
    [0, 0, 6, -3, 0],
    [0, 0, -3, 2, 0],
    [-2, 0, 0, 0, 5]
]

# Send and store the QUBO and retrieve the corresponding ID
qubo_id = luna.create_qubo(instance=tsp_instance)

In a similar fashion to when creating an instance of an optimization problem, Luna will store this problem in your account's database. You can access the QUBO for further computations just as you would with instances created through the use case library.

### Managing and accessing your optimization problems and solutions

All optimization problems created through Luna or uploaded to it will be stored in your account. You can always access a list of all IDs of the optimization problems you have created.

In [ ]:
# Maximum number of optimization problems to be returned
show_limit = 10

# Retrieve a list of your created optimization problem instances
all_optimization_ids = luna.get_all_optimizations(show_limit=show_limit)

You can use this functionality for repeated access at a later point, allowing you to create multiple solutions and perform various operations. Additionally, you have the ability to access individual instances to retrieve more detailed information, such as the name of the instance, its creation timestamp, or the specific QUBO formulation.

In [ ]:
# Select an instance you want more details about
optimization_instance_id = all_optimization_ids[0]

# Get more information about the specific instance
optimization_instance = luna.get_optimization(id=optimization_instance_id)

# Print the details of the specific instance
print(optimization_instance)

Furthermore, you have the option to rename the optimizations you have created.

In [ ]:
# Rename a specific instance
luna.rename_optimization(id=optimization_instance_id, name='New instance name')

You can always retrieve all existing solutions for any optimization.

In [ ]:
# Retrieve all solutions for a specific instance
all_solutions = luna.get_all_solutions(id=optimization_instance_id)

Or also retrieve any specific solution for your optimization problems.

In [ ]:
# Specify a solution ID to retrieve
solution_id = all_solutions[0]

# Retrieve all solutions for a specific instance
luna.get_solutions(optimization_id=optimization_instance_id, solution_id=solution_id)

Finally, if you no longer need a specific optimization, you can delete it from your account.

**Note:** Deleting an optimization will also delete all solutions corresponding to that optimization.

In [ ]:
# Delete a specific instance. Use with caution!
luna.delete_optimization(id=optimization_instance_id)

## Recommendation: Choosing the best Algorithm and Hardware


Luna provides you with a wide variety of preimplemented algorithms that can be used to solve your optimization problem. You can either select the algorithm yourself, or query our **Recommendation Engine** to retrieve a recommendation on the best algorithm to be used, together with a recommended set of parameters.  

Note that the current implementation of our recommendation engine only chooses between our available classical algorithms. New versions that will include quantum algorithms are already being developed.

In [ ]:
# Query the recommendation engine with our optimization instance
recommendation = ls.recommend(id=tsp_id)

# Investigate the recommendation from Luna
print(recommendation)

## Solve: Applying (quantum) algorithms to solve optimization problems

Overall, there are four different categories of algorithms available at the moment:

- **Quantum algorithms:** Algorithms that are run on real quantum hardware such as IBM or D-Wave.
- **Hybrid algorithms:** Algorithms that combine both quantum and classical hardware, often running certain computations on quantum hardware. These algorithms are often used to tackle the problem of the limited size of current quantum harwdare.
- **Quantum-inspired algorithms:** Algorithms that are fully run on classical hardware, but mimic certain properties of quantum algorithms during their computation. 
- **Classical algorithms:** Algorithms that are run on classical hardware and are used today to solve optimization problems.

You can either retrieve a recommendation using our Recommendation Engine from the previous chapter, or choose your algorithms and the corresponding hardware yourself. You will be able to find a list of all implemented algorithms on our [Solvers & Algorithms page](https://docs.aqarios.com/solvers-algorithms) as well as a list of all available quantum hardware providers' on our [Quantum Hardware page](https://docs.aqarios.com/quantum-hardware).

To solve an optimization problem, you can either use the recommendation returned from our Recommendation Engine, or simply choose the algorithm yourself. 

In [ ]:
# Solve the optimization instance with the recommendation
solver = recommendation['solver']
recommendation['params']
solution_url = luna.solve_problem(id=tsp_id, solver=solver, params=params)

# Or, alternatively, define your own solver to be used. Here, we use the classical heuristic Simulated Annealing
solver = "simulated_annealing"

# The parameters and their meanings can be found on our Solvers & Algorithms page on our documentation
params = {'beta_range': None,
         'num_reads': None,
         'num_sweeps': 200,
         'beta_schedule_type': 'geometric',
         'initial_states_generator': 'random',
         'n_init_samples': 1,
         'timeout': 5}

# When configuring the solver yourself, solving the problem still works the same way
solution_url = luna.solve_problem(id=tsp_id, solver=solver, params=params)

# When omitting parameters or the whole parameter set, the solver will fall back to the default parameter values found in our documentation
solution_url = luna.solve_problem(id=tsp_id, solver=solver)

Since some algorithms, especially those run on quantum hardware, can take some time to be executed, we don't want to keep you and the execution of your code waiting during this time. Instead, when a solution is requested, we will take care of all steps necesssary in an asynchronous fashion. This means that while your solution is being computed, you are free to do anything you want - and you can simply come back later, when the execution is finished.

In [ ]:
# After the execution of your algorithm has been finished, retrieve your raw solution
solution = ls.get_solution(solution_url)

# Print the raw solution
print(solution)

Note that this will return the raw solution that we get from the optimization algorithm that has been applied and which is dependent on the mathematical format that has been used. In case you are interested in the solution that has an interpretable meaning, there is one final step that needs to be done.

### Postprocess: Understanding raw outputs from optimization algorithms

As the raw output that we get from these algorithms is usually not too helpful if you are not familiar with optimization algorithms, we provide the possibility to map this raw output back into the domain of our initial problem formulation. 

Note that this feature only works for optimization problems generated through our use case library.

In [ ]:
# Apply postprocessing methods to get an interpretable solution
solution = ls.postprocess_solution(solution)

# Alternatively, you can directly retrieve the transformed solution
solution = ls.get_solution(solution_url, transform=True)

# Print the transformed solution 
print(solution)